In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense,Dropout,MaxPooling2D,Flatten,BatchNormalization,Conv2D,ZeroPadding2D
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import Adadelta,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
(x_train,y_train),(x_test,y_test)= cifar10.load_data()

In [3]:
x_train.shape

(50000, 32, 32, 3)

In [4]:

x_train.shape[1:]

(32, 32, 3)

In [5]:
num_classes = 10
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [6]:
#building the alex net architecture with 5 conv layers and 3 FC layers
l2_reg=0
model = Sequential()

#first conv layer
model.add(Conv2D(96,(11,11),input_shape = x_train.shape[1:],padding='same',kernel_regularizer=l2(l2_reg),activation='relu'))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#second conv layer
model.add(Conv2D(256,(5,5),padding='same',activation='relu'))
model.add(Dropout(0.3))#
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#third conv layer
#model.add(ZeroPadding2D((1,1)))
#model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
#model.add(Dropout(0.25))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))

#fourth conv layer
#model.add(ZeroPadding2D((1,1)))
#model.add(Conv2D(1024,(3,3),padding='same',activation='relu'))
#model.add(Dropout(0.3))
#model.add(BatchNormalization())
#model.add(MaxPooling2D(2,2))

#fifth conv layer
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(1024,(3,3),padding='same',activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))

#first FC layer
model.add(Flatten())
model.add(Dense(3072, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#second FC layer
#model.add(Dense(4096, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))

#third FC layer
model.add(Dense(num_classes , activation='softmax'))
model.add(BatchNormalization())

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 96)        34944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 96)        384       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       614656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 256)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 256)     

In [7]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.0001),metrics=['accuracy'])

In [8]:
#implementing callbacks
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint = ModelCheckpoint("cifar_alexnet_check1.h5",
                            monitor="val_loss",
                            mode="min",
                            save_best_only=True,
                            verbose=1)
early_stopping = EarlyStopping(monitor="val_loss",
                              min_delta=0,
                              patience=1,
                              verbose=1,
                              restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss",
                             factor=0.2,
                             patience=1,
                             verbose=1,
                             min_delta=0.0001)
#putting callbacks in callbacks list
callbacks = [checkpoint,early_stopping,reduce_lr]
model.compile(loss="categorical_crossentropy",
             optimizer=RMSprop(lr=0.0001),
             metrics=['accuracy'])


In [9]:
#batch_size = 32
#epoch = 5
#model.fit(x_train,y_train,batch_size=batch_size,epochs = epoch,
             #validation_data=(x_test,y_test),
             #shuffle=True)
        
nb_train_samples = 50000
nb__val_samples = 10000
batch_size=8
epochs = 5

model.fit(x_train,y_train,
                   steps_per_epoch=nb_train_samples//batch_size,
                   epochs=epochs,
                   callbacks=callbacks,
                   validation_data=(x_test,y_test),
                   validation_steps=nb__val_samples//batch_size)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 50000 samples, validate on 10000 samples
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[50000,32,32,96] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node conv2d_1/Relu}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
model.save("alexnet_cifar10_1.h5")